# Template

In [1]:
import gym
import numpy as np
import time
from IPython import display

In [2]:
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [4]:
env.observation_space.n

16

In [5]:
env.action_space.n

4

In [6]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [7]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 54)

In [8]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1)

(0.0, 4)

In [9]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, False)

(0.0, 6)

In [10]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, False)

(0.0, 18)

In [11]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [12]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 60/1000
Average number of steps: 12.033333333333333


In [14]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 100/1000
Average number of steps: 15.28


In [15]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 751/1000
Average number of steps: 36.806924101198405


In [16]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [17]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [19]:
np.all(v_values_1 >= v_values_0)

True

In [20]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [21]:
np.all(v_values_2 >= v_values_1)

True

In [22]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [23]:
np.all(v_values_3 >= v_values_2)

True

In [24]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)
    step = 0
    
    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            # print(f'Converged at {i}-th iteration.')
            step = i
            break

    return v_values, step

In [25]:
optimal_v_values, step = value_iteration(env, max_iters=500, gamma=0.9)

In [26]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [27]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [28]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [29]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 730/1000
Average number of steps: 37.25890410958904


In [30]:
play_multiple_times(env, policy_1, 1000)

Number of successes: 49/1000
Average number of steps: 11.040816326530612


# My code

## Policy Iteration

In [31]:
def policy_iteration(env, max_iters=500, gamma=0.9):
  """Policy Initialize"""
  pi_values = np.zeros(env.observation_space.n)
  step = 0  # number of steps to converge

  for i in range(max_iters):
    prev_pi_values = np.copy(pi_values)

    """Policy Evaluation"""
    v_values = policy_evaluation(env, pi_values)

    """Policy Improvement"""
    pi_values = policy_extraction(env, v_values)

    # Check exit condition
    if np.array_equal(pi_values, prev_pi_values):
      # print(f'Action converged at {i}-th iteration.')
      step = i
      break

  return pi_values, step

## FrozenLake-v1


In [62]:
# Create environment
env = gym.make('FrozenLake-v1')

In [63]:
frozenlake_value_iteration_runtime = []
frozenlake_policy_iteration_runtime = []

frozenlake_value_iteration_step = []
frozenlake_policy_iteration_step = []

### Value Iteration

In [64]:
for episode in range(50):
  start = time.time()

  # Value Iteration
  optimal_v_values, step = value_iteration(env, max_iters=500, gamma=0.9)

  # Policy Extraction
  optimal_pi_values = policy_extraction(env, optimal_v_values, gamma=0.9)

  end = time.time()

  # Test Policy
  # play_multiple_times(env, optimal_pi_values, max_episodes=1000)

  frozenlake_value_iteration_runtime.append(end - start)
  frozenlake_value_iteration_step.append(step)

### Policy Iteration

In [65]:
for episode in range(50):
  start = time.time()

  # Policy Iteration
  optimal_pi_values, step = policy_iteration(env, max_iters=500, gamma=0.9)

  end = time.time()

  # Test Policy
  # play_multiple_times(env, optimal_pi_values, max_episodes=1000)

  frozenlake_policy_iteration_runtime.append(end - start)
  frozenlake_policy_iteration_step.append(step)

In [66]:
print(f"Average runtime of Value Iteration: {np.average(frozenlake_value_iteration_runtime)} sec")

print(f"Average runtime of Policy Iteration: {np.average(frozenlake_policy_iteration_runtime)} sec")

Average runtime of Value Iteration: 0.04885705471038818 sec
Average runtime of Policy Iteration: 0.04803274631500244 sec


## FrozenLake8x8-v1

In [37]:
env = gym.make("FrozenLake8x8-v1")

In [38]:
frozenlake_8x8_value_iteration_runtime = []
frozenlake_8x8_policy_iteration_runtime = []

frozenlake_8x8_value_iteration_step = []
frozenlake_8x8_policy_iteration_step = []

### Value Iteration

In [39]:
for episode in range(50):
  start = time.time()

  # Value Iteration
  optimal_v_values, step = value_iteration(env, max_iters=500, gamma=0.9)

  # Policy Extraction
  optimal_pi_values = policy_extraction(env, optimal_v_values, gamma=0.9)

  end = time.time()

  # Test Policy
  # play_multiple_times(env, optimal_pi_values, max_episodes=1000)

  frozenlake_8x8_value_iteration_runtime.append(end - start)
  frozenlake_8x8_value_iteration_step.append(step)

### Policy Iteration

In [40]:
for episode in range(50):
  start = time.time()

  # Policy Iteration
  optimal_pi_values, step = policy_iteration(env, max_iters=500, gamma=0.9)

  end = time.time()

  # Test Policy
  # play_multiple_times(env, optimal_pi_values, max_episodes=1000)

  frozenlake_8x8_policy_iteration_runtime.append(end - start)
  frozenlake_8x8_policy_iteration_step.append(step)

In [41]:
print(f"Average runtime of Value Iteration: {np.average(frozenlake_8x8_value_iteration_runtime)} sec")

print(f"Average runtime of Policy Iteration: {np.average(frozenlake_8x8_policy_iteration_runtime)} sec")

Average runtime of Value Iteration: 0.3126011323928833 sec
Average runtime of Policy Iteration: 0.39844379901885985 sec


## Taxi-v3

In [42]:
env = gym.make('Taxi-v3')

In [43]:
taxi_value_iteration_runtime = []
taxi_policy_iteration_runtime = []

taxi_value_iteration_step = []
taxi_policy_iteration_step = []

### Value Iteration

In [44]:
for episode in range(50):
  start = time.time()

  # Value Iteration
  optimal_v_values, step = value_iteration(env, max_iters=500, gamma=0.9)

  # Policy Extraction
  optimal_pi_values = policy_extraction(env, optimal_v_values, gamma=0.9)

  end = time.time()

  # Test Policy
  # play_multiple_times(env, optimal_pi_values, max_episodes=1000)

  taxi_value_iteration_runtime.append(end - start)
  taxi_value_iteration_step.append(step)

### Policy Iteration

In [45]:
for episode in range(50):
  start = time.time()

  # Policy Iteration
  optimal_pi_values, step = policy_iteration(env, max_iters=500, gamma=0.9)

  end = time.time()

  # Test Policy
  # play_multiple_times(env, optimal_pi_values, max_episodes=1000)

  taxi_policy_iteration_runtime.append(end - start)
  taxi_policy_iteration_step.append(step)

In [46]:
print(f"Average runtime of Value Iteration: {np.average(taxi_value_iteration_runtime)} sec")

print(f"Average runtime of Policy Iteration: {np.average(taxi_policy_iteration_runtime)} sec")

Average runtime of Value Iteration: 2.501711354255676 sec
Average runtime of Policy Iteration: 4.593431344032288 sec


# Conclusion

In [49]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [67]:
import pandas as pd

data = {
    "Average Time (VI)": [np.average(frozenlake_value_iteration_runtime), 
                          np.average(frozenlake_8x8_value_iteration_runtime),
                          np.average(taxi_value_iteration_runtime)],
        
    "Average Iteration (VI)": [np.average(frozenlake_value_iteration_step), 
                          np.average(frozenlake_8x8_value_iteration_step),
                          np.average(taxi_value_iteration_step)],

    "Average Time (PI)": [np.average(frozenlake_policy_iteration_runtime), 
                          np.average(frozenlake_8x8_policy_iteration_runtime),
                          np.average(taxi_policy_iteration_runtime)],
        
    "Average Iteration (PI)": [np.average(frozenlake_policy_iteration_step), 
                          np.average(frozenlake_8x8_policy_iteration_step),
                          np.average(taxi_policy_iteration_step)],
}

df = pd.DataFrame(data, index = ['FrozenLake-v1', 'FrozenLake8x8-v1', 'Taxi-v3'])

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [68]:
# Trong các toy games, thuật toán Value Iteration (VI) cho ra kết quả nhanh hơn (trung bình thời gian 50 lần chạy) tại các game:
  # 1. FrozenLake8x8-v1 
  # 2. Taxi-v3
# Đặc biệt, với env Taxi-v3, Value Iteration chạy nhanh hơn Policy Iteration (PI) một cách rõ rệt (~2 lần).

print(df)

# Theo lý thuyết, Policy Iteration có độ phức tạp thời gian cho từng vòng lặp (O(S^2)) nhỏ hơn so với Value Iteration (O(S^2 + A)).
# Tuy nhiên, dựa trên kết quả thực nghiệm, PI mặc dù mất ít vòng lặp (iteration) để đi đến hội tụ hơn.
# Nhưng lại mất nhiều thời gian hơn để cho ra kết quả so với VI.

# Điều này có thể suy ra được rằng PI mặc dù mất ít vòng lặp hơn, nhưng lại tốn nhiều thời gian cho việc đánh giá từng chiến lược và
# cải thiện các chiến lược đó. 
# Đặc biệt đối với các bài toán với action space lớn (Taxi-v3), PI có thể gặp khó khăn trong việc tìm kiếm policy tối ưu. 

                  Average Time (VI)  Average Iteration (VI)  \
FrozenLake-v1              0.048857                    79.0   
FrozenLake8x8-v1           0.312601                   117.0   
Taxi-v3                    2.501711                   116.0   

                  Average Time (PI)  Average Iteration (PI)  
FrozenLake-v1              0.048033                     5.0  
FrozenLake8x8-v1           0.398444                     9.0  
Taxi-v3                    4.593431                    16.0  
